# File Name: simple_knwl_bases_retrival_langchain.ipynb
### Location: Chapter 7
### Purpose: 
#####             1. Use LangChain retrieve and generate integration with Amazon Bedrock. 
##### Dependency: simple_knwl_bases_building.ipynb at Chapter 7 should executed properly. 
# <ins>-----------------------------------------------------------------------------------</ins>

# <ins>Amazon SageMaker Classic</ins>
#### Those who are new to Amazon SageMaker Classic. Follow the link for the details. https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html

# <ins>Environment setup of Kernel</ins>
##### Fill "Image" as "Data Science"
##### Fill "Kernel" as "Python 3"
##### Fill "Instance type" as "ml-t3-medium"
##### Fill "Start-up script" as "No Scripts"
##### Click "Select"

###### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks-create-open.html for details.

# <ins>Mandatory installation on the kernel through pip</ins>

##### This lab will work with below software version. But, if you are trying with latest version of boto3, awscli, and botocore. This code may fail. You might need to change the corresponding api. 

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell. 

In [ ]:
%pip install --no-build-isolation --force-reinstall -q \
    "boto3>=1.34.84" \
    "opensearch-py>=2.7.1" \
    "retrying>=1.3.4" \
    "ragas" \
    "ipywidgets>=7.6.5" \
    "iprogress>=0.4" \
    "langchain>=0.2.16" \
    "langchain_community>=0.2.17" \
    "awscli>=1.32.84" \
    "botocore>=1.34.84" \
    "langchain-aws>=0.1.7" \
    "langchain-core"

# <ins>Disclaimer</ins>

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell.

# <ins>Restart the kernel</ins>

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

# <ins>Python package import</ins>

##### boto3 offers various clients for Amazon Bedrock to execute various actions.
##### botocore is a low-level interface to AWS tools, while boto3 is built on top of botocore and provides additional features

In [ ]:
import json
import os
import boto3
import botocore
import pprint
import random
from retrying import retry
import warnings
import time
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth, RequestError
from botocore.exceptions import NoCredentialsError, PartialCredentialsError
import pprint as pp
from botocore.exceptions import BotoCoreError, ClientError
from langchain_aws import ChatBedrock
from langchain.retrievers.bedrock import AmazonKnowledgeBasesRetriever
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

### Ignore warning 

In [ ]:
warnings.filterwarnings('ignore')

### %store magic command to retrive all the variable value from other notebook. 
### Here, simple_knwl_bases_building.ipynb

In [ ]:
%store -r

## Define important environment variable

In [ ]:
%%time

# Try-except block to handle potential errors
try:
    # Create a new Boto3 session to interact with AWS services
    boto3_session_name = boto3.session.Session()


    # Create a Bedrock Agent client using the current session and region
    bedrock_agent_client = boto3_session_name.client('bedrock-agent', region_name=aws_region_name)

    # Create an S3 client to interact with Amazon S3
    s3_client = boto3.client('s3')

    # Create an STS client to interact with AWS Security Token Service (STS)
    sts_client = boto3.client('sts')

    # Get the AWS account ID of the caller
    aws_account_id = sts_client.get_caller_identity()["Account"]


    # Create an OpenSearch Serverless (AOSS) client using the current session
    aoss_client = boto3_session_name.client('opensearchserverless')

    # Create boto3_bedrock_runtime_client
    boto3_bedrock_runtime_client = boto3.client('bedrock-runtime', region_name = aws_region_name)
    
    # Create an IAM client to interact with Identity and Access Management (IAM) service
    iam_client = boto3_session_name.client('iam')

    # Retrieve the current AWS account number and ARN of the caller
    sts_client = boto3.client('sts')
    identity_arn = sts_client.get_caller_identity().get('Arn')
    
    # Create boto3_bedrock_agent_runtime_client
    boto3_bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name=aws_region_name)
    
    # Store all variables in a dictionary
    variables_store = {
        "aws_region_name": aws_region_name,
        "bedrock_agent_client": bedrock_agent_client,
        "opensearch_service_name": opensearch_service_name,
        "s3_client": s3_client,
        "sts_client": sts_client,
        "aws_account_id": aws_account_id,
        "s3_suffix": s3_suffix,
        "s3_bucket_name": s3_bucket_name,
        "aoss_client": aoss_client,
        "vector_store_name": vector_store_name,
        "index_name": index_name,
        "iam_client": iam_client,
        "sts_client": sts_client,
        "identity_arn": identity_arn,
        "bedrock_knowledge_bases_name": bedrock_knowledge_bases_name,
        "aoss_collectionarn": aoss_collectionarn,
        "aoss_collection_host": aoss_collection_host,
        "genaibookedbedrocksagemakerexecutionrolearn": genaibookedbedrocksagemakerexecutionrolearn,
        "knowledgeBaseId": knowledgeBaseId
    }

    # Print all variables
    for var_name, value in variables_store.items():
        print(f"{var_name}: {value}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")


In [ ]:
# Define Model ID and prompt
# Main execution section
prompt = "What is Amazon doing and cashflow?"

# List of Bedrock models with names and model codes
bedrock_model_id = "anthropic.claude-3-haiku-20240307-v1:0"

#### Using LangChain framework


In this section, you explore the development of a Question & Answer (Q&A) application using the Retrieve API from Amazon Bedrock's Knowledge Bases in conjunction with LangChain. The application leverages the Anthropic Claude 3 Haiku model to provide answers to user queries.

You will begin by initializing the ChatBedrock model with the specified model ID and Bedrock client. Next, You will create an AmazonKnowledgeBasesRetriever object that interfaces with the Retrieve API. This retriever transforms your queries into embeddings, performs similarity searches within the knowledge base, and returns relevant document chunks. The output includes the retrieved text, source data URIs, and relevance scores, which allows for the construction of custom workflows based on semantic search results.

For example, when querying, "What is Amazon doing and cashflow?", the retriever is configured to return up to some relevant results, utilizing semantic search options for enhanced precision.

To tailor the model’s responses, you can define a system prompt that instructs the assistant on how to utilize the retrieved context effectively. You can use two helper functions to establish a processing chain:

    create_stuff_documents_chain: This function integrates the retrieved context directly into the prompt without additional summarization. It processes the input query along with the context to generate a concise answer.

    create_retrieval_chain: This function adds the retrieval step, allowing the model to output the context alongside the final answer.

Finally, the application invokes the retrieval chain with the user's query, efficiently generating and presenting the response. By employing LangChain's capabilities with Amazon Bedrock, we create a robust and flexible Q&A system that enhances the user's experience through accurate and contextually relevant answers.

In [ ]:
%%time

def initialize_llm(model_id: str, bedrock_client) -> ChatBedrock:
    """Initializes the ChatBedrock model."""
    try:
        llm = ChatBedrock(model_id=model_id, client=bedrock_client)
        return llm
    except Exception as e:
        print(f"Error initializing LLM: {e}")
        return None

def setup_retriever(kb_id: str) -> AmazonKnowledgeBasesRetriever:
    """Sets up the Amazon Knowledge Bases retriever."""
    try:
        retriever = AmazonKnowledgeBasesRetriever(
            knowledge_base_id=kb_id,
            retrieval_config={
                "vectorSearchConfiguration": {
                    "numberOfResults": 4,
                    "overrideSearchType": "SEMANTIC",  # optional
                }
            },
        )
        return retriever
    except Exception as e:
        print(f"Error setting up retriever: {e}")
        return None

def create_question_answer_chain(llm: ChatBedrock) -> create_stuff_documents_chain:
    """Creates a question-answering chain with the LLM."""
    system_prompt = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        "{context}"
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )
    
    try:
        question_answer_chain = create_stuff_documents_chain(llm, prompt)
        return question_answer_chain
    except Exception as e:
        print(f"Error creating question-answer chain: {e}")
        return None

def execute_rag_chain(retriever: AmazonKnowledgeBasesRetriever, question_answer_chain, query: str):
    """Executes the RAG chain and returns the response."""
    try:
        rag_chain = create_retrieval_chain(retriever, question_answer_chain)
        response = rag_chain.invoke({"input": query})
        return response
    except Exception as e:
        print(f"Error executing RAG chain: {e}")
        return None

def main(model_id: str, kb_id: str, bedrock_client, query: str):
    """Main function to run the retrieval-augmented generation."""
    llm = initialize_llm(model_id, bedrock_client)
    if llm is None:
        return

    retriever = setup_retriever(kb_id)
    if retriever is None:
        return

    question_answer_chain = create_question_answer_chain(llm)
    if question_answer_chain is None:
        return

    response = execute_rag_chain(retriever, question_answer_chain, query)
    if response:
        pp.pprint(response.get("answer", "No answer found."))

# Example usage
main(bedrock_model_id, knowledgeBaseId, boto3_bedrock_runtime_client, prompt)

# End of NoteBook 

#### <ins>Step 1</ins> 

##### Please ensure that you close the kernel after using this notebook to avoid any potential charges to your account.

##### Process: Go to "Kernel" at top option. Choose "Shut Down Kernel". 
##### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/studio-ui.html


#### <ins>Step 2</ins> 

#### If you are not executing any further lab of this Chapter 7
##### Execute the simple_knwl_bases_clean_up.ipynb to delete all the instances to avoid any potential charges to your account.